In [1]:
import warnings

import numpy as np
import pandas as pd
import sister
from tqdm import tqdm

In [2]:
warnings.filterwarnings('ignore')

In [3]:
data = pd.read_csv('/home/igor/PycharmProjects/bert-qa/data/squad_csv/train-squad.csv')
data = data[['context','question', 'id','text']]

In [4]:
data.drop_duplicates(inplace=True)

In [5]:
ids = []
word = []
target = []
words_in_context_with_ids = pd.DataFrame()
for _, v in tqdm(data.iterrows(), total=len(data)):
    try:
        words_in_context = set(v['context'].split())
        real_answer = set()
        real_answer.add(v['text'].split()[0])
        words_in_context = words_in_context.difference(real_answer)
        for w in words_in_context:
            ids.append(v['id'])
            word.append(w)
            target.append(0)
        ids.append(v['id'])
        word.append(real_answer.pop())
        target.append(1)
    except:
        continue


100%|██████████| 86821/86821 [00:28<00:00, 2994.97it/s]


In [6]:
words_in_context_with_ids = pd.DataFrame({'id': ids, "word": word, "target": target})


In [7]:
merged_df = pd.merge(data, words_in_context_with_ids, on=['id'], how='right')
merged_df = merged_df[['context', 'question', 'word','id', 'target']]


In [8]:
embedder = sister.MeanEmbedding(lang="en")


Loading model...


In [9]:
context_embeddins = []
question_embeddings = []
word_embeddigs = []

def get_embeddings(row):
    context_embeddins.append(embedder(row['context']))
    question_embeddings.append(embedder(row['question']))
    word_embeddigs.append(embedder(row['word']))


In [10]:
n_samples = 87000


In [11]:
merged_df_negative = merged_df[merged_df['target'] == 0]
merged_df_positive = merged_df[merged_df['target'] == 1]

merged_df_negative.reset_index(inplace=True)
merged_df_positive.reset_index(inplace=True)



merged_df_negative[0:n_samples].apply(get_embeddings, axis=1)



0        None
1        None
2        None
3        None
4        None
         ... 
86995    None
86996    None
86997    None
86998    None
86999    None
Length: 87000, dtype: object

In [12]:
context_embeddins = np.array(context_embeddins)
question_embeddings = np.array(question_embeddings)
word_embeddigs = np.array(word_embeddigs)


In [13]:
stacked = np.concatenate([context_embeddins, question_embeddings, word_embeddigs], axis=1)
embeddings_negative_df = pd.DataFrame(stacked)
embeddings_negative_df['target'] = merged_df_negative['target']
embeddings_negative_df['id'] = merged_df_negative['id']

embeddings_negative_df.to_csv('negative.csv', index=False)


In [14]:
context_embeddins = []
question_embeddings = []
word_embeddigs = []

merged_df_positive[0:n_samples].apply(get_embeddings, axis=1)
context_embeddins = np.array(context_embeddins)
question_embeddings = np.array(question_embeddings)
word_embeddigs = np.array(word_embeddigs)

stacked = np.concatenate([context_embeddins, question_embeddings, word_embeddigs], axis=1)
embeddings_positive_df = pd.DataFrame(stacked)
embeddings_positive_df['target'] = merged_df_positive['target']
embeddings_positive_df['id'] = merged_df_positive['id']

embeddings_positive_df.to_csv('positive.csv', index=False)
